In [1]:
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

from server import Server

import abc
import tensorflow as tf
import numpy as np

import tkinter
import _thread

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
# from tf_agents.environments import wrappers
# from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

tcl = tkinter.Tcl()
tcl.eval("package require Tk")

'8.6.10'

In [2]:
class CarMakerEnv(py_environment.PyEnvironment):

  def __init__(self):
    # set init state
    # set init obs_spec and action_spec

    while tcl.tk.eval("send CarMaker SimStatus") == "0":
      tcl.tk.eval("send CarMaker Appl::Start")
      print("send CarMaker StopSim")
    
    tcl.tk.eval("send CarMaker StartSim")
    print("send CarMaker StartSim")

    while tcl.tk.eval("send CarMaker SimStatus") != "0":
      print("send CarMaker SimStatus")
      self._state =  Server().server_step()

    self.last_time = -1
    self._episode_ended = False

    self._action_spec = array_spec.BoundedArraySpec(
        shape=(2,), dtype=np.float64, minimum=[-1., -9.425], maximum=[1., 9.425], name='action')

    self._observation_spec = array_spec.BoundedArraySpec(
      (14,), np.float64,  minimum=[0., -10., 
                                  -10., -3.14, 2., 0., 
                                  -10., -3.14, 2., 0., 
                                  -3.14, 0., 
                                  -3.14, 0.], 
                          maximum=[601, 100.,
                                  10., 3.14, 20., 999999., 
                                  10., 3.14, 20., 999999., 
                                  3.14, 999999., 
                                  3.14, 999999.], 
                          name='observation')


  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    print("Reset")
    # Restart TestRun and get initial state
    while tcl.tk.eval("send CarMaker SimStatus") == "0":
      tcl.tk.eval("send CarMaker Appl::Start")
    
    tcl.tk.eval("send CarMaker StartSim")

    while tcl.tk.eval("send CarMaker SimStatus") != "0":
      self._state = Server().server_step()

    self.last_state = {}
    self._episode_ended = False
    return ts.restart(self._state)

  def _step(self, action):

    # If TestRun end of time
    if self._state[0] >= 600 or self._state[0] == self.last_time:
      print("TestRun EOT")
      self._episode_ended = True
    else:
      self.last_time = self._state[0]
    
    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure that car is on route
    if self._state[4] < abs(self._state[2])*2:
      reward = -1000
      self._episode_ended = True
    else:
      self._state = Server().server_step(action)
      reward = self._state[1] * np.cos(self._state[3] * 10)

    

    if self._episode_ended:
      print("ts termination")
      self._reset()
      return ts.termination(self._state, reward)
    else:
      return ts.transition(
          self._state, reward, discount=0.9)

  

In [3]:
environment = CarMakerEnv()

utils.validate_py_environment(environment, episodes=5)

# time_step = environment.reset()
# print(time_step)
# cumulative_reward = time_step.reward

# vollgas = [1, 0]

# for _ in range(80):
#     time_step = environment.step(vollgas)
#     print(time_step)
#     cumulative_reward += time_step.reward


-1.35 0.02 -2.18 0.02
Sending string: -0.6654745328999034 5.645869475352336
Received request: 66.00 0.19 -6.82 -0.51 10.00 0.02 -6.82 -0.85 10.00 0.02 -1.35 0.02 -2.18 0.02
Sending string: 0.9828459531354095 0.8486069855175202
Received request: 66.12 0.41 -6.83 -0.51 10.00 0.02 -6.83 -0.85 10.00 0.02 -1.35 0.02 -2.18 0.02
Sending string: 0.28024064383361313 -6.2941074579658505
Received request: 66.25 0.34 -6.87 -0.52 10.00 0.02 -6.87 -0.85 10.00 0.02 -1.35 0.02 -2.19 0.02
Sending string: 0.33312497073881886 -7.217935201247188
Received request: 66.37 0.47 -6.93 -0.53 10.00 0.02 -6.93 -0.86 10.00 0.02 -1.36 0.02 -2.20 0.02
Sending string: -0.7585938115346949 -0.9626418542909949
Received request: 66.50 0.18 -6.95 -0.53 10.00 0.02 -6.95 -0.87 10.00 0.02 -1.37 0.02 -2.20 0.02
Sending string: -0.5612892339452504 0.49995149615308954
Received request: 66.62 0.05 -6.95 -0.53 10.00 0.02 -6.95 -0.87 10.00 0.02 -1.37 0.02 -2.20 0.02
Sending string: 0.13456519027478975 1.3940882764731217
Received r

TypeError: object of type 'NoneType' has no len()